visit original file: https://colab.research.google.com/drive/1h5VTF-Wqikfp2Hk-kud3lLB1Cs1iclXC?usp=sharing

In [1]:
!pip uninstall scikit-learn
!pip install scikit-learn==0.20.4
!pip install sklearn-porter

Found existing installation: scikit-learn 1.0.1
Uninstalling scikit-learn-1.0.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/scikit_learn-1.0.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scikit_learn.libs/libgomp-f7e03b3e.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-1.0.1
     |████████████████████████████████| 5.4 MB 4.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.20.4 which is incompatible.
     |████████████████████████████████| 144 kB 4.0 MB/s 


In [32]:
from sklearn.datasets import load_iris
from sklearn.tree import tree
from sklearn_porter import Porter
from enum import IntEnum

In [40]:
class PrivacyLevel(IntEnum):
  PUBLIC = 0
  NORMAL = 1
  PRIVATE = 2
  TOP_SECRET = 3

class ConnectionType(IntEnum):
  OFFLINE = 0
  WIFI = 1
  MOBILE = 2

class SignalStrenght(IntEnum):
  WEAK = 0
  NORMAL = 1
  STRONG = 2

class ConnectionSpeed(IntEnum):
  CRITICAL_LOW = 0
  LOW = 1
  NORMAL = 2
  FAST = 3
  ULTRA_FAST = 4

class EnviromentProperties:
  def __init__(self, 
               privacy_level: PrivacyLevel, 
               baterry_level: float, # [0, 1]
               signal_strenght: SignalStrenght,
               connection_type: ConnectionType,
               connection_speed: ConnectionSpeed,
               data_size: float, # KB
               use_cloud:bool = False
               ):
    self.privacy_level = privacy_level
    self.baterry_level = baterry_level
    self.signal_strenght = signal_strenght
    self.connection_type = connection_type
    self.connection_speed = connection_speed
    self.data_size = data_size
    self.use_cloud = 1 if use_cloud else 0

  def to_array(self):
    return [
            int(self.privacy_level), 
            self.baterry_level, 
            int(self.signal_strenght), 
            int(self.connection_type),
            int(self.connection_speed),
            self.data_size
            ]

In [41]:
data = [
  EnviromentProperties(
    privacy_level=PrivacyLevel.NORMAL,
    baterry_level=0.5,
    signal_strenght=SignalStrenght.NORMAL,
    connection_type=ConnectionType.WIFI,
    connection_speed=ConnectionSpeed.LOW,
    data_size=1000,
    use_cloud=False
  ), 
  EnviromentProperties(
    privacy_level=PrivacyLevel.NORMAL,
    baterry_level=0.1,
    signal_strenght=SignalStrenght.NORMAL,
    connection_type=ConnectionType.WIFI,
    connection_speed=ConnectionSpeed.FAST,
    data_size=1000,
    use_cloud=True
  ),
]

In [42]:
X = [item.to_array() for item in data]
Y = [item.use_cloud for item in data]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [43]:
print(clf.predict([
                   EnviromentProperties(
                    privacy_level=PrivacyLevel.NORMAL,
                    baterry_level=0.5,
                    signal_strenght=SignalStrenght.NORMAL,
                    connection_type=ConnectionType.WIFI,
                    connection_speed=ConnectionSpeed.FAST,
                    data_size=1000
                  ).to_array()
                  ]))

[0]


In [44]:
porter = Porter(clf, language='js')
output = porter.export(embed_data=True)
print(output)

var DecisionTreeClassifier = function() {

    var findMax = function(nums) {
        var index = 0;
        for (var i = 0; i < nums.length; i++) {
            index = nums[i] > nums[index] ? i : index;
        }
        return index;
    };

    this.predict = function(features) {
        var classes = new Array(2);
            
        if (features[1] <= 0.30000000074505806) {
            classes[0] = 0; 
            classes[1] = 1; 
        } else {
            classes[0] = 1; 
            classes[1] = 0; 
        }
    
        return findMax(classes);
    };

};

if (typeof process !== 'undefined' && typeof process.argv !== 'undefined') {
    if (process.argv.length - 2 === 6) {

        // Features:
        var features = process.argv.slice(2);

        // Prediction:
        var clf = new DecisionTreeClassifier();
        var prediction = clf.predict(features);
        console.log(prediction);

    }
}
